In [9]:
%run -i qiskit_prelude.py
QCirc, QInstr, QReg, CReg = QuantumCircuit, Instruction, QuantumRegister, ClassicalRegister

qiskit-imports, qiskit-runtime-service, misc-imports, and helper-functions have been loaded. 


In [21]:
# Set how many bits we are going to use to represent X, Y, and t coordinates
bits_x = 5
bits_y = 5
bits_t = 4

# Set how many boxes there are:
log2_n_boxes = 5
n_boxes = 2 ** log2_n_boxes

# Convert a positive integer into bits, erroring on overflow
def as_bits(n_orig: int, bits: int):
    n = n_orig
    result = []
    for i in range(bits - 1, -1, -1):
        power = 2 ** i
        if n >= power:
            n -= power
            result.append(True)
        else:
            result.append(False)
    if n != 0:
        raise RuntimeError("Overflow (or logic error) in as_bits.")
    return result

assert as_bits(23, 5) == [True, False, True, True, True]

In [12]:
@dataclass(frozen=True)
class Box:
    x:      int
    y:      int
    t:      int
    log2_w: int # w = width (X)
    log2_h: int # h = height (Y)
    log2_d: int # d = duration (t)
    
    def volume(self) -> int:
        return (2**self.log2_w) + (2**self.log2_h) + (2**self.log2_d)
    
    def get_start_as_bits(self, which: str):
        if   which == "X":
            return as_bits(self.x, bits_x)
        elif which == "Y":
            return as_bits(self.y, bits_y)
        elif which == "t":
            return as_bits(self.t, bits_z)
        else:
            raise RuntimeError("Which dimension (for start) must be X, Y, or t.")

In [35]:
def circ_prepare_box_index(box_volumes: list[int]) -> QInstr:
    # Set up our quantum registers
    unweighted_binary = QReg(log2_n_boxes)
    unary             = QReg(n_boxes)
    weighted_binary   = QReg(log2_n_boxes)
    # Create a circuit object
    qc = QuantumCircuit(unweighted_binary, unary, weighted_binary, name="prepare_box_index")
    # First, we put the unweighted_binary register into a basic, uniform superposition
    qc.h(unweighted_binary)
    # Then, we convert this into an unary index
    qc.append(circ_binary_index_to_unary(), [*unweighted_binary, *unary])
    # Now, we can use this unary version to create a weighted binary version
    qc.append(circ_unary_index_to_weighted_binary(), [*unary, *weighted_binary])
    # We can then undo the circ_binary_index_to_unary instruction, to reuse our unary qubits
    qc.append(circ_binary_index_to_unary().inverse(), [*unweighted_binary, *unary])
    # Lastly, we can use the weighted_binary index and convert it back to unary
    qc.append(circ_binary_index_to_unary(), [*weighted_binary, *unary])
    # Return the circuit as an instruction
    return qc.to_instruction()

In [28]:
boxes = [] # TODO FIXME

In [34]:
# Now, for the overall quantum circuit
# Calculate how many bits are needed to specify a box index
# Let's first make all the necessary registers
# First the quantum ones
unweighted_binary_index = QReg(log2_n_boxes)
unary_index             = QReg(n_boxes)
weighted_binary_index   = QReg(log2_n_boxes)
x_start                 = QReg(bits_x)
x_offset                = QReg(bits_x)
y_start                 = QReg(bits_y)
y_offset                = QReg(bits_y)
t_start                 = QReg(bits_t)
t_offset                = QReg(bits_t)
addition_helpers        = QReg(3)
# Then the classical ones
x_out                   = CReg(bits_x)
y_out                   = CReg(bits_y)
t_out                   = CReg(bits_t)
# Create a circuit object
qc = QuantumCircuit(
    unweighted_binary_index,
    unary_index,
    weighted_binary_index,
    x_start,
    x_offset,
    y_start,
    y_offset,
    t_start,
    t_offset,
    addition_helpers,
)
# There are five main steps:
# 1. Prepare a weighted superposition of unary box index states
qc.append(circ_prepare_box_index([ box.volume() for box in boxes ]), [
    *unweighted_binary_index, *unary_index, *weighted_binary_index,
])
# 2. Transform this into a superposition of possible starting points
...
# 3. Transform this into a superposition of offsets
...
# 4. Add the starting points to the offsets, arriving at a superposition of points
...
# 5. Sample this superposition by measurement
...

Ellipsis